# QMMM workflow using GROMACS and VOTCA-XTP

## What is this tutorial about
In this tutorial, we will learn how to set and perform excited state calculation using the Votca XTP library. We will use methane as our QM region.

## Requirements
* You will need to install **VOTCA** using the instructions described [here](https://github.com/votca/votca/blob/master/share/sphinx/INSTALL.rst)
* Once the installation is completed you need to activate the VOTCA enviroment by running the `VOTCARC.bash` script that has been installed at the bin subfolder for the path that you have provided for the installation step above

## Interacting with the XTP command line interface
The XTP package offers the following command line interface that the user can interact with:
* [xtp_map](https://www.votca.org/xtp/xtp_map.html)
* [xtp_parallel](https://www.votca.org/xtp/xtp_parallel.html)
* [xtp_run](https://www.votca.org/xtp/xtp_run.html)
* [xtp_tools](https://www.votca.org/xtp/xtp_tools.html)

Run the following command to view the help message of `xtp_tools`:

In [1]:
!xtp_tools -h

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_tools, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Runs excitation/charge transport tools



Allowed options:
  -h [ --help ]                 display this help and exit
  --verbose                     be loud and noisy
  --verbose1                    be very loud and noisy
  -v [ --verbose2 ]             be extremly loud and noisy
  -o [ --options ] arg          Tool user options.
  -t [ --nthreads ] arg (=1)    number of threads to create
  -e [ --execute ] arg        Name of Tool to run
  -l [ --list ]               Lists all available Tools
  -d [ --description ] arg    Short description of a Tools
  -c [ --cmdoptions ] arg     Modify options via command line by e.g. '-c 
                              xmltag.subtag=value'. Use whitespace to separate 
                              multiple o

### Note
> * In Jupyter the `!` symbol means: *run the following command as a standard unix command*
> * In Jupyter the command `%env` set an environmental variable

## Setting the environment


Remove previous hdf5 file

In [2]:
!rm -f state.hdf5

## Generate the topology from the Gromacs file

runs the mapping from MD coordinates to segments and creates an [hdf5 file](https://www.hdfgroup.org/solutions/hdf5/). You can explore the generated `state.hdf5` file with e.g. hdf5itebrowser. In Python, you can use the [h5py library](https://www.h5py.org/).

In [3]:
!xtp_map -t MD_FILES/topol.tpr -c MD_FILES/conf.gro -s system.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_map, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Reading file MD_FILES/topol.tpr, VERSION 5.1.1 (single precision)
Note: file tpx version 103, software tpx version 137


Creating statefile state.hdf5
Frame with id 0 was not in statefile state.hdf5 ,adding it now.


Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


##  Check the mapping

Let us first output `.pdb` files for the segments, qmmolecules and classical segments in order to check the mapping. Use `xtp_run -d mapchecker` to see all options `mapchecker` calculator takes. We use the `-c` option to change one option on the commandline.

In the [mapchecker section of the manual](https://votca.github.io/xtp/mapchecker.html) you can find a table with the `mapchecker` input variables and their corresponding defaults. Finally, the following command run the check

In [4]:
!xtp_run -e mapchecker -c map_file=system.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_run, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... mapchecker
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... mapchecker
 Using 1 threads
Writing segments to md_segments_step_0.pdb
Writing qmmolecules to qm_segments_n_step_0.pdb


Writing polarsegments to mp_segments_e_step_0.pdb


Writing polarsegments to mp_segments_h_step_0.pdb


Changes have not been written to state file.


## Neighborlist Calculation

The following step is to determine the neighbouring pairs for exciton transport. See the [neighborlist options](https://www.votca.org/xtp/neighborlist.html) for further information. 

Finally, we can run the calculation using 4 threads

In [5]:
!xtp_run -e neighborlist -c exciton_cutoff=0.5 constant=0.6 -f state.hdf5 -t 4

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_run, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... neighborlist
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... neighborlist
 Using 4 threads
Evaluating 1000 segments for neighborlist. 
 ... ... Evaluating 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
*****************************************

**********

 ... ... Created 21093 direct pairs.
 ... ... Determining classical pairs 
 ... ... Found 8586 classical pairs 


Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


## Read reorganization energies
In this step we will read the in site reorganization energies and store them in the `state.hdf5` file. We just need to copy the input file and execute the calculation. The side energies have to be calculated by the user beforehand and put into an xml file. We added them to `system.xml`

In [6]:
!xtp_run -e einternal -c energies_file=system.xml -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_run, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... einternal
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... einternal
 Using 1 threads
... ... Site, reorg. energies from system.xml.

... ... Read in site, reorg. energies for 1000 segments. 

Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


## Compute site energy
In this step we will perform some *QMMM* calculations to compute the site energies. The `qmmm_mm.xml` file contains some predefined settings to perform the *MM* calculations. Let us first copy these settings into the state file. Instead of using the `-c` option we now use the `-o` option to read in options from an xml file.

In [7]:
!xtp_parallel -e qmmm -o qmmm_mm.xml -f state.hdf5 -j "write"

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... qmmm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... qmmm 
... ... Writing job file qmmm_mm_jobs.xml


... ... In total 4000 jobs
Changes have not been written to state file.


The previous command generates a `qmmm_mm_jobs.xml` containing 4000 *MM* jobs to compute, if you examine that file, it should look something like

```xml
<jobs>
  <job>
    <id>0</id>
    <tag>Methane_0:n</tag>
    <input>
      <site_energies>0:n</site_energies>
      <regions>
        <region>
          <id>0</id>
          <segments>0:n</segments>
        </region>
      </regions>
    </input>
    <status>AVAILABLE</status>
  </job>    
...
```

Let us run just the first 4 jobs by settings all jobs `status` to `COMPLETE` except for the first four. This can be easily done with [sed](https://www.gnu.org/software/sed/manual/sed.html) as follows,

In [8]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_mm_jobs.xml

Now we can run the jobs and save the results in the state file

In [9]:
!xtp_parallel -e qmmm -o qmmm_mm.xml -f state.hdf5 -x 2 -j "run"
!xtp_parallel -e qmmm -o qmmm_mm.xml -f state.hdf5 -j "read"

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... qmmm

... ... Initialized with 1 threads.

... ... Using 2 openmp threads for 1x2=2 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... qmmm 


MST ERR Job file = 'qmmm_mm_jobs.xml', cache size =  8
MST ERR Initialize jobs from qmmm_mm_jobs.xml


MST ERR Registered 4000 jobs.
T00 ERR ... Requesting next job


T00 ERR ... Assign jobs from stack

...


Changes have not been written to state file.


========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... qmmm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 


... qmmm 
Found 1 states of type e
Found 1 states of type h
Found 1 states of type s
Found 1 states of type n
3996 incomplete jobs found.


Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


## Site energy and pair energy analysis
In this step we generate an histogram and compute the correlation function of site energies and pair energy differences.

In [10]:
!xtp_run -e eanalyze -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_run, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... eanalyze
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... eanalyze
 Using 1 threads
... ... Short-listed 1000 segments (pattern='*')
... ... ... NOTE Statistics of site energies and spatial correlations thereof are based on the short-listed segments only. 
... ... ...      Statistics of site-energy differences operate on the full list.
... ... excited state e


... ... excited state h


... ... excited state s


... ... excited state t


Changes have not been written to state file.


You should now see a set of files prefixed with `eanalyze` containing the histrogram and correlation functions.

In [11]:
!ls eanalyze*

eanalyze.pairhist_e.out  eanalyze.pairlist_s.out  eanalyze.sitehist_e.out
eanalyze.pairhist_h.out  eanalyze.pairlist_t.out  eanalyze.sitehist_h.out
eanalyze.pairhist_s.out  eanalyze.sitecorr_e.out  eanalyze.sitehist_s.out
eanalyze.pairhist_t.out  eanalyze.sitecorr_h.out  eanalyze.sitehist_t.out
eanalyze.pairlist_e.out  eanalyze.sitecorr_s.out
eanalyze.pairlist_h.out  eanalyze.sitecorr_t.out


## QM energy calculation
Our next task is to perform the qm calculations for each segment that we have stored in the hdf5 file. The calculations take place in 3 stages: write the jobs to a file, perform the computation and finally save the results to the state file. We provided a small options file to make the computation cheaper.

In [12]:
!cat eqm.xml

<?xml version="1.0"?>
<options>
  <eqm help="Executes qm calculations for individual molecules" section="sec:eqm">
    <map_file>system.xml</map_file>
    <gwbse>
      <gw>
        <mode>G0W0</mode>
      </gw>
      <bse>
        <exctotal>10</exctotal>
      </bse>
      <ranges>full</ranges>
    </gwbse>
    <dftpackage>
      <basisset>3-21G</basisset>
      <auxbasisset>aux-def2-svp</auxbasisset>
    </dftpackage>
  </eqm>
</options>

We set the GWBSE mode to `G0W0`,  the `ranges` to `full` and the `basisset` and `auxbasisset` to `3-21G` and `aux-def2-svp`. For more information, check the [eqm calculator options](https://votca.github.io/xtp/eqm.html). For the sake of computational time let just compute the `gw` approximation and the `singlet`. You can also request the `triplet` or `all`

First we will write the job in a file and enable only the first 2,

In [13]:
!xtp_parallel -e eqm -o eqm.xml -f state.hdf5 -s 0 -j "write"
!sed -i "s/AVAILABLE/COMPLETE/g" eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' eqm.jobs

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... eqm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... eqm 
... ... Writing job file: eqm.jobs

 with 1000 jobs
Changes have not been written to state file.


Now, let run these 2 jobs

In [14]:
!xtp_parallel -e eqm -o eqm.xml -f state.hdf5 -x 2 -s 0 -j run -q 1

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... eqm

... ... Initialized with 1 threads.

... ... Using 2 openmp threads for 1x2=2 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... eqm 
MST ERR Job file = 'eqm.jobs', cache size =  1
MST ERR Initialize jobs from eqm.jobs


MST ERR Registered 1000 jobs.
T00 ERR ... Requesting next job
T00 ERR ... Assign jobs from stack


T00 ERR ... Next job: ID = 0=> [ 0%] 
T00 ERR ... 2025-11-3 4:22:10 Evaluating site 0
T00 ERR ... Running DFT


T00 ERR ... Running GWBSE

...


T00 ERR ... Running GWBSE


T00 ERR ... Running ESPFIT
T00 ERR ... ===== Running on 2 threads ===== 


T00 ERR ... 2025-11-3 4:22:15 Calculated Densities at Numerical Grid, Number of electrons is 2.24147e-08
T00 ERR ... 2025-11-3 4:22:15 Calculating ESP at CHELPG grid points


T00 ERR ... 2025-11-3 4:22:16 Netcharge constrained to -0
T00 ERR ...  Sum of fitted charges: -2.22045e-16
T00 ERR ...  RMSE of fit:  0.00114403
T00 ERR ...  RRMSE of fit: 0.0642009
T00 ERR ... El Dipole from fitted charges [e*bohr]:
		 dx = -0.5613 dy = -0.1878 dz = -0.6118 |d|^2 = +0.7245
T00 ERR ... El Dipole from exact qm density [e*bohr]:
		 dx = -0.5741 dy = -0.1919 dz = -0.6258 |d|^2 = +0.7581
T00 ERR ... Written charges to MP_FILES/frame_0/n2s1/Methane_1_n2s1.mps
T00 ERR ... 2025-11-3 4:22:16 Finished evaluating site 1
T00 ERR ... Saving data to molecule_1.orb
T00 ERR ... Reporting job results

T00 ERR ... Requesting next job
T00 ERR ... Assign jobs from stack
T00 ERR ... Sync did not yield any new jobs.
T00 ERR ... Next job: ID = - (none available)


MST ERR Assign jobs from stack
Changes have not been written to state file.


## QM calculation for pairs
In the following step we will run QM calculations for each pair in the hdf5 file. As the calculations on the previous step, we will first write the jobs in a file, then run them and finally store the results in the state file. First, we need to copy the input to our local folder

As in the previous section, we set the GWBSE mode to `G0W0`, the `ranges` to `full` and the `basisset` and `auxbasisset` to `3-21G` and `aux-def2-svp`. But we compute only the `gw` approximation, as the BSE is formed in the coupling step only once and we do not have to diagonalize it. For more information, check the [iqm calculator options](https://www.votca.org/xtp/iqm.html). We only compute the `singlet` couplings. 

Before running the calculations, we need to specify in the `iqm` input which states to read into the jobfile for each segment type.

In [15]:
!cat iqm.xml

<?xml version="1.0"?>
<options>
  <iqm help="Executes qm calculations for individual molecules" section="sec:eqm">
    <map_file>system.xml</map_file>
    <gwbse>
      <tasks>gw</tasks>
      <gw>
        <mode>G0W0</mode>
      </gw>
      <ranges>full</ranges>
    </gwbse>
    <dftpackage>
      <basisset>3-21G</basisset>
      <auxbasisset>aux-def2-svp</auxbasisset>
    </dftpackage>
    <bsecoupling>
      <spin>singlet</spin>
    </bsecoupling>
    <readjobfile help="which states to read into the jobfile for each segment type">
      <singlet>Methane:s1</singlet>
      <triplet>Methane:t1</triplet>
      <electron>Methane:e1</electron>
      <hole>Methane:h1</hole>
    </readjobfile>
  </iqm>
</options>

Now, let's write the jobs to the file

In [16]:
!xtp_parallel -e iqm -o iqm.xml -f state.hdf5 -s 0 -j "write"

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... iqm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 


... iqm 
... ... Writing job file iqm.jobs


... ... In total 12507 jobs
Changes have not been written to state file.


From the jobs that we just write down, let's make available only the first job

In [17]:
!sed -i "s/AVAILABLE/COMPLETE/g" iqm.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' iqm.jobs

Now we can run and store the jobs results

In [18]:
!xtp_parallel -e iqm -o iqm.xml -f state.hdf5 -x 2 -s 0 -j run -q 1

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... iqm

... ... Initialized with 1 threads.

... ... Using 2 openmp threads for 1x2=2 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... iqm 
MST ERR Job file = 'iqm.jobs', cache size =  1
MST ERR Initialize jobs from iqm.jobs


MST ERR Registered 12507 jobs.
T00 ERR ... Requesting next job


T00 ERR ... Assign jobs from stack


T00 ERR ... Next job: ID = 0=> [ 0%] 
T00 ERR ... 2025-11-3 4:22:18 Evaluating pair 0 [0:1] out of 21093
T00 ERR ... Running DFT

...


T00 ERR ... 2025-11-3 4:22:21 Finished evaluating pair 0:1
T00 ERR ... Orb file is not saved according to options 
T00 ERR ... Reporting job results

T00 ERR ... Requesting next job


T00 ERR ... Assign jobs from stack


T00 ERR ... Sync did not yield any new jobs.
T00 ERR ... Next job: ID = - (none available)


MST ERR Assign jobs from stack


Changes have not been written to state file.


Finally, we read the results into the state

In [19]:
!xtp_parallel -e iqm -o iqm.xml -f state.hdf5 -j "read"

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... iqm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 


... iqm 


 ERROR    Pairs [total:updated(e,h,s,t)] 21093:(1,1,1,0) Incomplete jobs: 12506



Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


## Coupling
We can now compute the classical coupling of transition in the aformentioned three stages,

We need to change in the `iexcitoncl` input the name `map_file` option and add the state. check all the available of the [iexcitoncl calculator](https://www.votca.org/xtp/iexcitoncl.html). We do this via the commandline using the `-c` option.

In [20]:
!xtp_parallel -e iexcitoncl -c map_file=system.xml states=Methane:n2s1 -f state.hdf5 -j "write"

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... iexcitoncl

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... iexcitoncl 
... ... Writing job file exciton.jobs


... ... In total 8586 jobs
Changes have not been written to state file.


In [21]:
!head -n 15 exciton.jobs

<jobs>
	<job>
		<id>0</id>
		<tag></tag>
		<input>
			<segment id="0" mps_file="MP_FILES/Methane_n2s1.mps" type="Methane">0</segment>
			<segment id="2" mps_file="MP_FILES/Methane_n2s1.mps" type="Methane">2</segment>
		</input>
		<status>AVAILABLE</status>
	</job>
	<job>
		<id>1</id>
		<tag></tag>
		<input>
			<segment id="0" mps_file="MP_FILES/Methane_n2s1.mps" type="Methane">0</segment>


Now we can run and save the jobs. For demo purposes we will run only the first job

In [22]:
!sed -i "s/AVAILABLE/COMPLETE/g" exciton.jobs
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' exciton.jobs
!xtp_parallel -e iexcitoncl -c map_file=system.xml states=Methane:n2s1 -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... iexcitoncl

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... iexcitoncl 
MST ERR Job file = 'exciton.jobs', cache size =  8
MST ERR Initialize jobs from exciton.jobs


MST ERR Registered 8586 jobs.
T00 ERR ... Requesting next job


T00 ERR ... Assign jobs from stack


T00 ERR ... Next job: ID = 0=> [ 0%] 
T00 ERR ... 2025-11-3 4:22:24 Evaluating pair 0 [0:2]
T00 ERR ... Reporting job results

T00 ERR ... Requesting next job


T00 ERR ... Assign jobs from stack


T00 ERR ... Sync did not yield any new jobs.
T00 ERR ... Next job: ID = - (none available)


MST ERR Assign jobs from stack


Changes have not been written to state file.


In [23]:
!xtp_parallel -e iexcitoncl -c map_file=system.xml -f state.hdf5 -j "read"

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... iexcitoncl

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... iexcitoncl 


 ERROR    Neighborlist size 21093
 ERROR    Pairs in jobfile [total:updated:incomplete] 8586:1:8585


Wrote MD topology (step = 0, time = 0) to state.hdf5
... . 


## Coupling analysis
Using the coupling computed in the previous steps, we will generate an histogram for the squared couplings in logarithmic scale,

In [24]:
!xtp_run -e ianalyze -c states=e,h,s -f state.hdf5

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_run, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... ianalyze
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... ianalyze
 Using 1 threads
Calculating for state e now.
Calculating for state h now.
Calculating for state s now.
Changes have not been written to state file.


## QMMM calculations

Finally let us run a proper qmmm calculation using the qmmm calculator

In [25]:
!cat qmmm.xml

<?xml version="1.0"?>
<options>
  <qmmm help="Executes qmmm calculations for individual molecules and clusters" section="sec:qmmm">
    <print_regions_pdb help="print the geometry of the regions to a pdb file">true</print_regions_pdb>
    <max_iterations help="max iterations for qmmm scf loop">50</max_iterations>
    <map_file help="xml file with segment definition">system.xml</map_file>
    <job_file help="name of jobfile to which jobs are written">qmmm_jobs.xml</job_file>
    <io_jobfile>
    <states>n s1 t1</states>
    </io_jobfile>
    <regions>
      <qmregion help="definition of a region">
        <id help="id of a region has to start from 0">0</id>
        <gwbse>
          <gw>
            <mode>G0W0</mode>
          </gw>
          <bse>
            <exctotal>10</exctotal>
          </bse>
          <ranges>full</ranges>
        </gwbse>
        <dftpackage>
          <basisset>3-21G</basisset>
          <auxbasisset>aux-def2-svp</auxbasisset>
        </dftpackage>
        <s

In [26]:
!xtp_parallel -e qmmm -o qmmm.xml -f state.hdf5 -j "write"

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... qmmm

... ... Initialized with 1 threads.

... ... Using 1 openmp threads for 1x1=1 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 


... qmmm 
... ... Writing job file qmmm_jobs.xml


... ... In total 3000 jobs
Changes have not been written to state file.


Lets run just the first job

In [27]:
!sed -i "s/AVAILABLE/COMPLETE/g" qmmm_jobs.xml                                                                                                                                                            
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_jobs.xml   
!sed -i '0,/COMPLETE/s/COMPLETE/AVAILABLE/' qmmm_jobs.xml  
!xtp_parallel -e qmmm -o qmmm.xml -x 2 -f state.hdf5 -j run

========   VOTCA (http://www.votca.org)   ========

please read and cite: https://doi.org/10.21105/joss.06864
and submit bugs to https://github.com/votca/votca/issues

xtp_parallel, version 2025.1-dev gitid: 789c479 (compiled Nov  3 2025, 04:19:59)

Initializing calculator
... qmmm

... ... Initialized with 1 threads.

... ... Using 2 openmp threads for 1x2=2 total threads.
1 frames in statefile, Ids are: 0 
Starting at frame 0
Evaluating frame 0
Import MD Topology (i.e. frame 0) from state.hdf5


.... 
... qmmm 


MST ERR Job file = 'qmmm_jobs.xml', cache size =  8
MST ERR Initialize jobs from qmmm_jobs.xml


MST ERR Registered 3000 jobs.
T00 ERR ... Requesting next job


T00 ERR ... Assign jobs from stack

...


T00 ERR ...  Region:qmregion 0 is converged deltaE=8.250861896e-07 RMS Dmat=6.674983904e-06 MaxDmat=3.834895703e-05
T00 ERR ... 2025-11-3 4:23:1 Region:polarregion 1 is converged deltaE=8.901880267e-08
T00 ERR ... 2025-11-3 4:23:1 --Total Energy all regions -39.72067327
T00 ERR ... 2025-11-3 4:23:1 Job converged after 13 iterations.
T00 ERR ... Next State is: s1
T00 ERR ... Reporting job results

T00 ERR ... Requesting next job


T00 ERR ... Assign jobs from stack


T00 ERR ... Sync did not yield any new jobs.
T00 ERR ... Next job: ID = - (none available)


MST ERR Assign jobs from stack


Changes have not been written to state file.


Finally, save the results. We could read them in but that is a bit pointless. Maybe check out how to turn a checkpoint file into an or orbfile (look at the scripts) and visualise it with the `gencube` tool. 

In [28]:
#!xtp_parallel -e qmmm -o OPTIONFILES/qmmm.xml -f state.hdf5 -j "read"